# LangChain 1.0 Basic(Real World) Agent in Databricks

Ref:
https://docs.langchain.com/oss/python/langchain/quickstart#build-a-real-world-agent

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 mlflow

%restart_python

## モデルとの接続

In [0]:
from langchain.chat_models import init_chat_model
import mlflow
import os

mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
model = init_chat_model(
    # "openai:databricks-llama-4-maverick",
    "openai:databricks-qwen3-next-80b-a3b-instruct",
    api_key=creds.token,
    base_url=creds.host + "/serving-endpoints",
)


## 単純なエージェントの作成と実行

In [0]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

SYSTEM_PROMPT = """あなたはダジャレを交えた話し方をする天気予報の専門家です。

あなたは2つのツールにアクセスできます:

- get_weather_for_location: 特定の場所の天気を取得するために使用します
- get_user_location: ユーザーの現在地を取得するために使用します

ユーザーが天気を尋ねてきた場合、必ず場所を把握してください。質問からユーザーの現在地を指していると判断できる場合は、get_user_locationツールを使って場所を特定してください。
"""

@tool
def get_weather_for_location(city: str) -> str:
    """指定した都市の天気を取得します。"""
    return f"{city}はいつも晴れです！"


@dataclass
class Context:
    """カスタム実行時コンテキストスキーマ。"""
    user_id: str


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """ユーザーIDに基づいてユーザー情報（場所）を取得します。"""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [0]:
from dataclasses import dataclass

# ここではdataclassを使用していますが、Pydanticモデルもサポートされています。
@dataclass
class ResponseFormat:
    """エージェントのレスポンススキーマ"""

    # ダジャレを含むレスポンス（必須）
    punny_response: str
    # 天気に関する興味深い情報（あれば）
    weather_conditions: str | None = None

In [0]:
from langgraph.checkpoint.memory import InMemorySaver

# 短期記憶をメモリ内に保持
checkpointer = InMemorySaver()

In [0]:
from langchain.agents import create_agent
from pprint import pprint

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# `thread_id` は会話ごとの一意な識別子です。
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "外の天気はどうですか？"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

# 同じ `thread_id` を使って会話を続けることができます。
response = agent.invoke(
    {"messages": [{"role": "user", "content": "ありがとう！"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])